# Data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
from IPython.display import Image, display

In the prevous section, we described hedge funds, and more generally the buy-side industry, as `consumers of data`. In practice hedge funds have been a beneficiary of the exponential growth in the number of available datasets. We first describe the main classes of datasets that have been made available in the industry. We then introduce some of the illustrative `toy` datasets that will be used across notebooks. 

## Data deluge

There are now sensors everywhere in the physical world and most of the online interactions are tracked -- leading a `data deluge` (e.g. see [Mary Meeker (2018)](http://www.kpcb.com/internet-trends) on internet trends or [Lori Lewis (2022)](https://www.allaccess.com/merge/archive/31294/infographic-what-happens-in-an-internet-minute)). On the one hand, the cost of sensors has dropped so much that these sensors can be put practically everywhere and record data from the physicial world. On the other hand, the volume and velocity of data only has also exploded. 

In [ ]:
# hide
# sidebyside
display(
    Image("images/sensors.PNG", width=500),
    Image("images/internet_minute_2020.png", width=400),
)

### Data scouting

The article "Hedge funds see a gold rush is data mining" [FT (08/28/2017)](https://www.ft.com/content/d86ad460-8802-11e7-bf50-e1c239b45787) describes use-cases associated with four types of alternative data: 

- web traffic 

- credit card transaction

- geolocalisation 

- satellite imaging 

In each of these cases, the main idea is to use alternative data to get a noisy, but real-time forecast of company fundamentals (e.g. quarterly  sales). 

In [ ]:
# hide
display(Image("images/ftalternativedata.png", width=700))

Obviously, predicting company fundamentals is only part of the story of what drives stock prices in financial markets. On the other side, understanding the market participants, in particular retail traders, is also important as shown in China [Bloomberg (06/15/2019)](https://www.bloomberg.com/news/articles/2019-05-15/quants-think-like-amateurs-in-the-world-s-wildest-stock-market) or in the US with the Game Stop saga [FT(01/29/2021)](https://www.ft.com/content/04477ee8-0af2-4f0f-a331-2987444892c3). 

In [ ]:
# hide
# sidebyside
display(
    Image("images/bloomberg_investing_in_china.png", width=600),
    Image("images/game_stop.png", width=600),
)

More recent examples are more sophisticated and creativity for where to look is key. 

In [ ]:
# hide
display(Image("images/ft_sec_as_data.png", width=600))

### ML research

ML research has become a race with new ideas coming out with an increasing speed -- e.g. as illustrated by the number of papers published on the scientific paper repository arxiv.com ([Jeff Dean (06/02/2019)](https://twitter.com/JeffDean/status/1135114657344237568)). 

In [ ]:
# hide
display(Image("images/mlarxiv2.png", width=600))

The success of deep-learning depends on: i) model capacity, ii) computational power, iii) dataset  size. [Sun, Shrivastava, Singh, Gupta (2017)](https://arxiv.org/abs/1707.02968) note that the size of the largest dataset has remained somewhat constant over the last few years. 

In [ ]:
# hide
# sidebyside
display(
    Image("images/unreasonableEffectiveness.PNG", width=300),
    Image("images/unreasonableEffectiveness2.PNG", width=300),
)

Here are several trends from the Stanford's AI index report: 

- The training costs for cutting-edge AI models have skyrocketed, with OpenAI's GPT-4 and Google's Gemini Ultra costing \\$78m and \\$191m, respectively.
- AI projects on GitHub have surged from 845 in 2011 to about 1.8m in 2023, with a 59.3% increase in 2023 alone.
- As language models have rapidly advanced, surpassing human performance on numerous benchmarks, there has been a growing need for more robust and wide-ranging evaluation methods -- with the Holistic Evaluation of Language Models (HELM) providing a comprehensive framework .
-  Introduced by OpenAI researchers in 2021, HumanEval is a benchmark for assessing the coding abilities of AI systems. A variant of the GPT-4 model, AgentCoder, is currently leading in HumanEval performance. 

In [ ]:
# hide
display(Image("images/nlp_model_size2.png", width=800))

In [ ]:
# hide
display(Image("images/nlp_model_cost.png", width=800))

In [ ]:
# hide
display(Image("images/ai_understanding.png", width=800))

In [ ]:
# hide
display(Image("images/ai_adoption.png", width=800))

## Datasets

In particular, we look at several sources of data:

- Ken French's data library

- Berkshire Hathaway financials 

- Stock returns 

- Federal Open Market Committee (FOMC) statements

- the sentiment dictionary created by Loughran and McDonalds 

- US filings to the U.S. Securities and Exchange Commission (SEC) called 10Qs and 10Ks

- Long term stock market data put together by Amit Goyal

### data helper functions

In this section, we use the `jupyter` command `%%writefile` to construct a module of data helper function. 

In [ ]:
%%writefile ../skfin/datasets.py
import logging
import os
import re
import subprocess
import sys
import warnings
from io import BytesIO
from zipfile import ZipFile

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


from skfin.data_utils import clean_directory_path, load_dict, save_dict
from skfin.dataset_dates import load_fomc_change_date
from skfin.dataset_mappings import mapping_10X, symbol_dict

#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)


def clean_kf_dataframes(df, multi_df=False):
    """
    extract the annual and monthly dataframes from the csv file with specific formatting
    """
    idx = [-2] + list(np.where(df.notna().sum(axis=1) == 0)[0])
    if multi_df:
        cols = ["  Average Value Weighted Returns -- Monthly"] + list(
            df.loc[df.notna().sum(axis=1) == 0].index
        )
    returns_data = {"Annual": {}, "Monthly": {}}
    for i in range(len(idx)):
        if multi_df:
            c_ = (
                cols[i]
                .replace("-- Annual", "")
                .replace("-- Monthly", "")
                .strip()
                .replace("/", " ")
                .replace(" ", "_")
            )
        if i != len(idx) - 1:
            v = df.iloc[idx[i] + 2 : idx[i + 1] - 1].astype(float)
            v.index = v.index.str.strip()
            if len(v) != 0:
                if len(v.index[0]) == 6:
                    v.index = pd.to_datetime(v.index, format="%Y%m")
                    if multi_df:
                        returns_data["Monthly"][c_] = v
                    else:
                        returns_data["Monthly"] = v
                    continue
                if len(v.index[0]) == 4:
                    v.index = pd.to_datetime(v.index, format="%Y")
                    if multi_df:
                        returns_data["Annual"][c_] = v
                    else:
                        returns_data["Annual"] = v
    return returns_data


def load_kf_returns(
    filename="12_Industry_Portfolios", cache_dir="data", force_reload=False
):
    """
    industry returns from Ken French:
    https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
    """

    if filename == "12_Industry_Portfolios":
        skiprows, multi_df = 11, True
    if filename == "F-F_Research_Data_Factors":
        skiprows, multi_df = 3, False
    if filename == "F-F_Momentum_Factor":
        skiprows, multi_df = 13, False
    if filename == "F-F_Research_Data_Factors_daily":
        skiprows, multi_df = 4, False

    output_dir = clean_directory_path(cache_dir) / filename
    if (output_dir.is_dir()) & (~force_reload):
        logger.info(f"logging from cache directory: {output_dir}")
        returns_data = load_dict(output_dir)
    else:
        logger.info("loading from external source")
        path = (
            "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/"
            + filename
            + "_CSV.zip"
        )
        r = requests.get(path)
        files = ZipFile(BytesIO(r.content))

        df = pd.read_csv(files.open(filename + ".CSV"), skiprows=skiprows, index_col=0)
        if "daily" in filename:
            returns_data = {
                "Daily": df.iloc[:-1].pipe(
                    lambda x: x.set_index(pd.to_datetime(x.index))
                )
            }
        else:
            returns_data = clean_kf_dataframes(df, multi_df=multi_df)

        logger.info(f"saving in cache directory {output_dir}")
        save_dict(returns_data, output_dir)
    return returns_data


def load_buffets_data(cache_dir="data", force_reload=False):
    """
    data from Stephen Lihn: site: https://github.com/slihn
    """
    filename = clean_directory_path(cache_dir) / "ffdata_brk13f.parquet"

    if (filename.is_file()) & (~force_reload):
        logger.info(f"logging from cache directory: {filename}")
        df = pd.read_parquet(filename)

    else:
        logger.info("loading from external source")
        path = "https://github.com/slihn/buffetts_alpha_R/archive/master.zip"
        r = requests.get(path) #, verify=False)
        files = ZipFile(BytesIO(r.content))

        df = pd.read_csv(
            files.open("buffetts_alpha_R-master/ffdata_brk13f.csv"), index_col=0
        )
        df.index = pd.to_datetime(df.index, format="%m/%d/%Y")
        logger.info(f"saving in cache directory {filename}")
        df.to_parquet(filename)
    return df


def load_sklearn_stock_returns(cache_dir="data", force_reload=False):
    """
    data from scikit-learn
    """
    filename = clean_directory_path(cache_dir) / "sklearn_returns.parquet"
    if (filename.is_file()) & (~force_reload):
        logger.info(f"logging from cache directory: {filename}")
        df = pd.read_parquet(filename)

    else:
        logger.info("loading from external source")
        url = "https://raw.githubusercontent.com/scikit-learn/examples-data/master/financial-data"
        df = (
            pd.concat(
                {
                    c: pd.read_csv(f"{url}/{c}.csv", index_col=0, parse_dates=True)[
                        "close"
                    ].diff()
                    for c in symbol_dict.keys()
                },
                axis=1,
            )
            .asfreq("B")
            .iloc[1:]
        )

        logger.info(f"saving in cache directory {filename}")
        df.to_parquet(filename)
    return df


def get_fomc_urls(from_year=1999, switch_year=None):
    if switch_year is None:
        from datetime import datetime

        switch_year = datetime.now().year - 5
    calendar_url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
    r = requests.get(calendar_url)
    soup = BeautifulSoup(r.text, "html.parser")
    contents = soup.find_all(
        "a", href=re.compile("^/newsevents/pressreleases/monetary\d{8}[ax].htm")
    )
    urls_ = [content.attrs["href"] for content in contents]

    for year in range(from_year, switch_year):
        yearly_contents = []
        fomc_yearly_url = (
            f"https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"
        )
        r_year = requests.get(fomc_yearly_url)
        soup_yearly = BeautifulSoup(r_year.text, "html.parser")
        yearly_contents = soup_yearly.findAll("a", text="Statement")
        for yearly_content in yearly_contents:
            urls_.append(yearly_content.attrs["href"])

    urls = ["https://www.federalreserve.gov" + url for url in urls_]
    return urls


def sent_cleaner(s):
    return s.replace("\n", " ").replace("\r", " ").replace("\t", " ").strip()


def bs_cleaner(bs, html_tag_blocked=None):
    if html_tag_blocked is None:
        html_tag_blocked = [
            "style",
            "script",
            "[document]",
            "meta",
            "a",
            "span",
            "label",
            "strong",
            "button",
            "li",
            "h6",
            "font",
            "h1",
            "h2",
            "h3",
            "h5",
            "h4",
            "em",
            "body",
            "head",
            "sup",
        ]
    return [
        sent_cleaner(t)
        for t in bs.find_all(text=True)
        if (t.parent.name not in html_tag_blocked) & (len(sent_cleaner(t)) > 0)
    ]


regexp = re.compile(r"\s+", re.UNICODE)


def feature_extraction(corpus, sent_filters=None):
    if sent_filters is None:
        sent_filters = [
            "Board of Governors",
            "Federal Reserve System",
            "20th Street and Constitution Avenue N.W., Washington, DC 20551",
            "Federal Reserve Board - Federal Reserve issues FOMC statement",
            "For immediate release",
            "Federal Reserve Board - FOMC statement",
            "DO NOT REMOVE:  Wireless Generation",
            "For media inquiries",
            "or call 202-452-2955.",
            "Voting",
            "For release at",
            "For immediate release",
            "Last Update",
            "Last update",
        ]

    text = [
        " ".join(
            [
                regexp.sub(" ", s)
                for i, s in enumerate(c)
                if (i > 1) & np.all([q not in s for q in sent_filters])
            ]
        )
        for c in corpus
    ]

    release_date = [pd.to_datetime(c[1].replace("Release Date: ", "")) for c in corpus]
    last_update = [
        pd.to_datetime(
            [
                s.replace("Last update:", "").replace("Last Update:", "").strip()
                for s in c
                if "last update: " in s.lower()
            ][0]
        )
        for c in corpus
    ]
    voting = [" ".join([s for s in c if "Voting" in s]) for c in corpus]
    release_time = [
        " ".join(
            [s for s in c if ("For release at" in s) | ("For immediate release" in s)]
        )
        for c in corpus
    ]

    return pd.DataFrame(
        {
            "release_date": release_date,
            "last_update": last_update,
            "text": text,
            "voting": voting,
            "release_time": release_time,
        }
    )


def load_fomc_statements(
    add_url=True, cache_dir="data", force_reload=False, progress_bar=False, from_year=1999
):
    """
    https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm
    """
    filename = clean_directory_path(cache_dir) / "fomc_statements.parquet"
    if (filename.exists()) & (~force_reload):
        logger.info(f"logging from cache file: {filename}")
        statements = pd.read_parquet(filename)
    else:
        logger.info("loading from external source")
        urls = get_fomc_urls(from_year=from_year)
        if progress_bar:
            urls_ = tqdm(urls)
        else:
            urls_ = urls
        corpus = [
            bs_cleaner(BeautifulSoup(requests.get(url).text, "html.parser"))
            for url in urls_
        ]
        statements = feature_extraction(corpus).set_index("release_date")
        if add_url:
            statements = statements.assign(url=urls)
        statements = statements.sort_index()
        logger.info(f"saving cache file {filename}")
        statements.to_parquet(filename)
    return statements


def load_loughran_mcdonald_dictionary(cache_dir="data", force_reload=False):
    """
    Software Repository for Accounting and Finance by Bill McDonald
    https://sraf.nd.edu/loughranmcdonald-master-dictionary/
    """
    filename = (
        clean_directory_path(cache_dir)
        / "Loughran-McDonald_MasterDictionary_1993-2021.csv"
    )
    if (filename.exists()) & (~force_reload):
        logger.info(f"logging from cache file: {filename}")
    else:
        logger.info("loading from external source")
        id = "17CmUZM9hGUdGYjCXcjQLyybjTrcjrhik"
        url = f"https://docs.google.com/uc?export=download&confirm=t&id={id}"        
        subprocess.run(f"wget -O '{filename}' '{url}'", shell=True, capture_output=True)
    return pd.read_csv(filename)


def load_10X_summaries(cache_dir="data", force_reload=False):
    """
    Software Repository for Accounting and Finance by Bill McDonald
    https://sraf.nd.edu/sec-edgar-data/
    """
    filename = (
        clean_directory_path(cache_dir)
        / "Loughran-McDonald_10X_Summaries_1993-2021.csv"
    )
    if (filename.is_file()) & (~force_reload):
        logger.info(f"logging from cache directory: {filename}")
    else:
        logger.info("loading from external source")
        id = "1CUzLRwQSZ4aUTfPB9EkRtZ48gPwbCOHA"
        url = f"https://docs.google.com/uc?export=download&confirm=t&id={id}"
        subprocess.run(f"wget -O '{filename}' '{url}'", shell=True, capture_output=True)

    df = pd.read_csv(filename).assign(
        date=lambda x: pd.to_datetime(x.FILING_DATE, format="%Y%m%d")
    )
    return df


def load_ag_features(cache_dir="data", sheet_name="Monthly", force_reload=False):
    """
    load features from Amit Goyal's website:
    https://sites.google.com/view/agoyal145
    """
    filename = clean_directory_path(cache_dir) / "PredictorData2021.xlsx"
    if (filename.exists()) & (~force_reload):
        logger.info(f"logging from cache file: {filename}")
    else:
        id = "1OArfD2Wv9IvGoLkJ8JyoXS0YMQLDZfY2"
        url = f"https://docs.google.com/uc?export=download&confirm=t&id={id}"
        subprocess.run(f"wget -O '{filename}' '{url}'", shell=True, capture_output=True)
    return (
        pd.read_excel(filename, sheet_name=sheet_name)
        .assign(date=lambda x: pd.to_datetime(x.yyyymm, format="%Y%m"))
        .set_index("date")
    )

### Ken French data: industry returns

In [ ]:
from skfin.datasets import (
    load_buffets_data,
    load_kf_returns,
    load_sklearn_stock_returns,
)

In [ ]:
%%time
returns_data = load_kf_returns(filename="12_Industry_Portfolios", force_reload=True)

Reloading from a cache directory is faster!

In [ ]:
%%time
returns_data = load_kf_returns(filename="12_Industry_Portfolios", force_reload=False)

In [ ]:
returns_data_SMB_HML = load_kf_returns(filename="F-F_Research_Data_Factors")

In [ ]:
returns_data_MOM = load_kf_returns(filename="F-F_Momentum_Factor")

In [ ]:
returns_data_DAILY = load_kf_returns(filename="F-F_Research_Data_Factors_daily", force_reload=True)["Daily"]

### Stock returns (2003-2007)

In [ ]:
%%time
returns_data = load_sklearn_stock_returns(force_reload=True)

In [ ]:
from skfin.datasets import symbol_dict
from skfin.metrics import sharpe_ratio
from skfin.plot import bar

In [ ]:
start_date, end_date = returns_data.index[0].strftime("%Y-%m-%d"), returns_data.index[
    -1
].strftime("%Y-%m-%d")

df = (
    returns_data.pipe(sharpe_ratio)
    .rename(index=symbol_dict)
    .sort_values()
    .pipe(lambda x: pd.concat([x.head(), x.tail()]))
)
bar(
    df,
    horizontal=True,
    title=f"Annualized stock sharpe ratio: {start_date} to {end_date}",
)

### 13F Berkshire Hathaway

In [ ]:
#import os; os.environ['REQUESTS_CA_BUNDLE']='/etc/ssl/certs/ca-bundle.crt'

In [ ]:
%%time
df = load_buffets_data(force_reload=True)

### FOMC Statements 

In [ ]:
from skfin.datasets import load_fomc_statements

In [ ]:
%%time
statements = load_fomc_statements(force_reload=True)

In [ ]:
%%time
statements = load_fomc_statements(force_reload=False)

### Loughran-McDonalds sentiment dictionary and regulartory filing summaries

In [ ]:
from skfin.datasets import load_loughran_mcdonald_dictionary, load_10X_summaries

In [ ]:
%%time
filing_summaries = load_10X_summaries(force_reload=False)

In [ ]:
%%time
lm = load_loughran_mcdonald_dictionary(force_reload=False)

### Goyal long-term market data

In [ ]:
from skfin.datasets import load_ag_features

In [ ]:
%%time 
ag = load_ag_features(force_reload=False)